# Deep Q-Learning

In [1]:
%matplotlib inline
import gym
from gym.wrappers import Monitor
import itertools
import numpy as np
import os
import random
import sys
import tensorflow as tf
from collections import deque, namedtuple
import matplotlib.pyplot as plt
from utils import EpisodeStats

In [2]:
env = gym.envs.make('Breakout-v0')

[2017-11-04 16:42:30,135] Making new env: Breakout-v0


### Actions

In [3]:
print(env.env.get_action_meanings())

['NOOP', 'FIRE', 'RIGHT', 'LEFT']


In [4]:
VALID_ACTIONS = [0,1,2,3]

### Preprocessing Environment State

raw Atari 2600 frames: 210x160 pixel images with a 128-colour palette


**basic preprocessing step:**
* 4 last screen images
* resize to 84x84
* convert to grayscale (256 gray levels)

$256^{84x84x4} \approx 10^{67970}$ possible game states

In [5]:
class StateProcessor():
    def __init__(self):
        # build TF graph
        with tf.variable_scope('state_processor'):
            self.input_state = tf.placeholder(shape=[210, 160, 3], dtype=tf.uint8)
            self.output = tf.image.rgb_to_grayscale(self.input_state)
            self.output = tf.image.crop_to_bounding_box(
                self.output, 34, 0, 160, 160)
            self.output = tf.image.resize_images(
                self.output, [84, 84], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
            self.output = tf.squeeze(self.output)
    
    def process(self, sess, state):
        """
        Args:
            sess: TF session object
            state: [210, 160, 3] Atari RGB State
        Returns:
            processed [84,84,1] state
        """
        return sess.run(self.output, { self.input_state: state})

## Q-Network

**Network Architecture:**

|Layer|Input   |Filter size|Stride|Num filters|Activation|Output  |
|-----|--------|-----------|------|-----------|----------|--------|
|conv1|84x84x4 |8x8        |4     |32         |ReLU      |20x20x32|
|conv2|20x20x32|4x4        |3     |64         |ReLU      |9x9x64  |
|conv3|9x9x64  |3x3        |1     |64         |ReLU      |7x7x64  |
|fc4  |7x7x64  |           |      |512        |ReLU      |512     |
|fc5  |512     |           |      |18         |Linear    |18*     |

\* number of valid actions

**Optimizer RMSProp:**
* lr = 0.00025
* decay = 0.99
* no momentum
* epsilon = 1e-6

### Estimator

* Estimator is used for both Q-Network and the Target Network

In [6]:
class Estimator():
    def __init__(self, scope='estimator', summaries_dir=None):
        self.scope = scope
        # write Tensorboard summaries to disk
        self.summary_writer = None
        with tf.variable_scope(scope):
            self._build_model()
            if summaries_dir:
                summary_dir = os.path.join(summaries_dir, 'summaries_{}'.format(scope))
                if not os.path.exists(summary_dir):
                    os.makedirs(summary_dir)
                self.summary_writer = tf.summary.FileWriter(summary_dir)
    
    def _build_model(self):
        # input are 4 RGB frames w/ shape 160x160
        self.X = tf.placeholder(shape=[None, 84, 84, 4], dtype=tf.uint8, name='X')
        # TD-target value
        self.y = tf.placeholder(shape=[None], dtype=tf.float32, name='y')
        # int id for selected actions
        self.actions = tf.placeholder(shape=[None], dtype=tf.int32, name='actions')
        X = tf.to_float(self.X)/255.0
        batch_size = tf.shape(self.X)[0]
        # Network
        conv1 = tf.contrib.layers.conv2d(X, 32, 8, 4, activation_fn=tf.nn.relu)
        conv2 = tf.contrib.layers.conv2d(conv1, 64, 4, 3, activation_fn=tf.nn.relu)
        conv3 = tf.contrib.layers.conv2d(conv2, 64, 3, 1, activation_fn=tf.nn.relu)
        fc4 = tf.contrib.layers.fully_connected(
            tf.contrib.layers.flatten(conv3), 512)
        self.preds = tf.contrib.layers.fully_connected(fc4, len(VALID_ACTIONS))
        # predictions for chosen actions only
        gather_indices = tf.range(batch_size)*tf.shape(self.preds)[1]+self.actions
        self.action_preds = tf.gather(tf.reshape(self.preds, [-1]), gather_indices)
        # calculate the loss
        self.losses = tf.squared_difference(self.y, self.action_preds)
        self.loss = tf.reduce_mean(self.losses)
        # Optimizer
        self.optimizer = tf.train.RMSPropOptimizer(0.00025, decay=0.99, epsilon=1e-6)
        self.train_op = self.optimizer.minimize(
            self.loss, global_step=tf.contrib.framework.get_global_step())
        # summaries for Tensorboard
        self.summaries = tf.summary.merge([
            tf.summary.scalar('loss', self.loss),
            tf.summary.histogram('loss_hist', self.losses),
            tf.summary.histogram('q_values_hist', self.preds),
            tf.summary.scalar('max_q_value', tf.reduce_max(self.preds))
        ])
    
    def predict(self, sess, s):
        """
        Args:
            sess: TF session object
            s: state input of shape [batch_size, 4, 160, 160, 3]
        
        Returns:
            tensor of shape [batch_size, NUM_VALID_ACTIONS] containing estimated action values
        """
        return sess.run(self.preds, {self.X: s})
    
    def update(self, sess, s, a, y):
        """
        Updates estimator towards given targets (y)
        Args:
            sess: TF session object
            s: state input of shape [batch_size, 4, 160, 160, 3]
            a: chosen action of shape [batch_size]
            y: targets of shape [batch_size]
        Returns:
            calculated loss on the batch
        """
        feed_dict = {self.X: s, self.y: y, self.actions: a}
        summaries, global_step, _, loss = sess.run([
            self.summaries,
            tf.contrib.framework.get_global_step(),
            self.train_op,
            self.loss
        ], feed_dict)
        if self.summary_writer:
            self.summary_writer.add_summary(summaries, global_step)
        return loss

#### Test

In [7]:
tf.reset_default_graph()
global_step = tf.Variable(0, name='global_step', trainable=False)

e = Estimator(scope='test')
sp = StateProcessor()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # example observation batch
    observation = env.reset()
    
    observation_p = sp.process(sess, observation)
    observation = np.stack([observation_p]*4, axis=2)
    observations = np.array([observation]*2)
    # test prediction
    print(e.predict(sess, observations))
    # test training step
    y = np.array([10.0, 10.0])
    a = np.array([1, 3])
    print(e.update(sess, observations, a, y))
    

/home/mark/.conda/envs/dl/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[ 0.          0.04459491  0.0059645   0.00791901]
 [ 0.          0.04459491  0.0059645   0.00791901]]
99.4759


In [8]:
def copy_model_parameters(sess, estimator1, estimator2):
    """
    Copies model parameters from one estimator to another.
    Args:
        sess: TF session instance
        estimator1: Estimator to copy parameters from
        estimator2: Estimator to copy parameters to
    """
    e1_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator1.scope)]
    e1_params = sorted(e1_params, key=lambda v: v.name)
    e2_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator2.scope)]
    e2_params = sorted(e2_params, key=lambda v: v.name)
    
    update_ops = []
    for e1_v, e2_v in zip(e1_params, e2_params):
        op = e2_v.assign(e1_v)
        update_ops.append(op)
    
    sess.run(update_ops)

### $\epsilon$-greedy policy

In [9]:
def epsilon_greedy(estimator, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximator and epsilon.
    Args:
        estimator: An estimator that returns q values for a given state
        nA: Number of actions in the environment.
    Returns:
        A function that takes the (sess, observation, epsilon) as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.

    """
    def policy_fn(sess, observation, epsilon):
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = estimator.predict(sess, np.expand_dims(observation, 0))[0]
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

### Q-Learning

In [10]:
def deep_q_learning(sess,
                    env,
                    q_estimator,
                    target_estimator,
                    state_processor,
                    n_episodes,
                    experiments_dir,
                    replay_mem_size=500000,
                    replay_mem_init_size=10000,
                    estimator_update_steps=10000,
                    discount_factor=0.99,
                    epsilon_start=1.0,
                    epsilon_end=0.1,
                    epsilon_decay_steps=500000,
                    batch_size=32,
                    record_video=50):
    """
    Q-Learning algorithm for TD control using Function Approximation
    Finds optimal greedy policy while following epsilon-greedy policy
    Args:
        sess: TF session object
        env: OpenAI env
        q_estimator: Estimator object used for the q values
        target_estimator: Estimator object used for targets
        state_processor: StateProcessor object
        n_episodes: Number of episodes to run
        experiments_dir: Dir to save TF summaries in
        replay_mem_size: Size of replay memory
        replay_mem_init_size: Number of random experiences to sample
                            when initializing the replay memory
        estimator_update_steps: Copy params from q_estimator to target_estimator every N steps
        discount_factor: Lambda time discount factor
        epsilon_start: Chance to sample a random action when taking action (decayed over time)
        epsilon_end: Final minimum value of epsilon after decay
        batch_size: Size of batches to sample from replay memory
        record_video: Record a video every N steps
    Returns:
        EpisodeStats object
    """
    Transition = namedtuple('Transition', ['state', 'action', 'reward', 'next_state', 'done'])
    
    # the replay memory
    replay_memory = []
    
    # keep track of useful stats
    stats = EpisodeStats(
        episode_lengths=np.zeros(n_episodes),
        episode_rewards=np.zeros(n_episodes))
    
    # create dirs for checkpoints and summaries
    checkpoint_dir = os.path.join(experiments_dir, 'checkpoints')
    checkpoint_path = os.path.join(checkpoint_dir, 'model')
    monitor_path = os.path.join(experiments_dir, 'monitor')
    
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(monitor_path):
        os.makedirs(monitor_path)
    
    saver = tf.train.Saver()
    
    # load previous checkpoint if it exists
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print('Loading model checkpoint {}...\n'.format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)
    
    # get current time step
    total_t = sess.run(tf.contrib.framework.get_global_step())
    
    # epsilon decay schedule
    epsilons = np.linspace(epsilon_start, epsilon_end, epsilon_decay_steps)
    
    policy = epsilon_greedy(q_estimator, len(VALID_ACTIONS))
    
    # populate replay memory with initial experience
    print('Populating replay memory...')
    state = env.reset()
    state = state_processor.process(sess, state)
    state = np.stack([state]*4, axis=2)
    for i in range(replay_mem_init_size):
        a_probs = policy(sess, state, epsilons[min(total_t, epsilon_decay_steps-1)])
        action = np.random.choice(np.arange(len(a_probs)), p=a_probs)
        next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
        next_state = state_processor.process(sess, next_state)
        next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
        replay_memory.append(Transition(state, action, reward, next_state, done))
        if done:
            state = env.reset()
            state = state_processor.process(sess, state)
            state = np.stack([state]*4, axis=2)
        else:
            state = next_state
    
    # record videos
    env = Monitor(env, directory=monitor_path,
                  resume=True, video_callable=lambda count: count%record_video == 0)
    
    for i_episode in range(n_episodes):
        # save current checkpoint
        saver.save(tf.get_default_session(), checkpoint_path)
        
        # reset the environment
        state = env.reset()
        state = state_processor.process(sess, state)
        state = np.stack([state]*4, axis=2)
        loss = None
        
        for t in itertools.count():
            # epsilon for this time step
            epsilon = epsilons[min(total_t, epsilon_decay_steps-1)]
            
            # add epsilon to Tensorboard
            episode_summary = tf.Summary()
            episode_summary.value.add(simple_value=epsilon, tag='epsilon')
            q_estimator.summary_writer.add_summary(episode_summary, total_t)
            
            # update target estimator
            if total_t%estimator_update_steps == 0:
                copy_model_parameters(sess, q_estimator, target_estimator)
                print('\n Copied model parameters to target network')
            
            # print step information
            print('\rStep {} ({}) @ Episode {}/{}, loss: {}'.format(
                t, total_t, i_episode+1, n_episodes, loss), end='')
            sys.stdout.flush()
            
            # take step in environment
            a_probs = policy(sess, state, epsilon)
            action = np.random.choice(np.arange(len(a_probs)), p=a_probs)
            next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
            next_state = state_processor.process(sess, next_state)
            next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
            
            # if replay memory is full, pop first element
            if len(replay_memory) == replay_mem_size:
                replay_memory.pop(0)
            
            # save transition to replay memory
            replay_memory.append(Transition(state, action, reward, next_state, done))
            
            # update stats
            stats.episode_lengths[i_episode] = t
            stats.episode_rewards[i_episode] += reward
            
            # sample minibatch from the replay memory
            samples = random.sample(replay_memory, batch_size)
            s_batch, a_batch, r_batch, next_s_batch, done_batch = map(np.array, zip(*samples))
            # calc q values and targets (Double DQN)
            q_values_next = q_estimator.predict(sess, next_s_batch)
            best_actions = np.argmax(q_values_next, axis=1)
            q_values_next_target = target_estimator.predict(sess, next_s_batch)
            targets_batch = r_batch+np.invert(done_batch).astype(np.float32)*\
                discount_factor*q_values_next_target[np.arange(batch_size), best_actions]
            # perform GD update
            s_batch = np.array(s_batch)
            loss = q_estimator.update(sess, s_batch, a_batch, targets_batch)
            
            if done:
                break
            
            state = next_state
            total_t += 1
            
        # add summaries to Tensorboard
        episode_summary = tf.Summary()
        episode_summary.value.add(
            simple_value=stats.episode_rewards[i_episode],
            node_name='episode_reward',
            tag='episode_reward')
        episode_summary.value.add(
            simple_value=stats.episode_lengths[i_episode],
            node_name='episode_length',
            tag='episode_length')
        q_estimator.summary_writer.add_summary(episode_summary, total_t)
        q_estimator.summary_writer.flush()

        yield total_t, EpisodeStats(
            episode_lengths=stats.episode_lengths[:i_episode+1],
            episode_rewards=stats.episode_rewards[:i_episode+1])
        
    env.monitor.close()
    return stats

In [ ]:
tf.reset_default_graph()
experiments_dir = os.path.abspath('./experiments/{}'.format(env.spec.id))
# create global step variable
global_step = tf.Variable(0, name='global_step', trainable=False)
# estimators
q_estimator = Estimator(scope='q', summaries_dir=experiments_dir)
target_estimator = Estimator(scope='target_q')
# state processor
state_processor = StateProcessor()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t, stats in deep_q_learning(sess,
                                    env,
                                    q_estimator=q_estimator,
                                    target_estimator=target_estimator,
                                    state_processor=state_processor,
                                    n_episodes=10000,
                                    experiments_dir=experiments_dir,
                                    replay_mem_size=500000,
                                    replay_mem_init_size=50000,
                                    estimator_update_steps=10000,
                                    discount_factor=0.99,
                                    epsilon_start=1.0,
                                    epsilon_end=0.1,
                                    epsilon_decay_steps=500000,
                                    batch_size=32):
        
        print('\nEpisode Reward: {}'.format(stats.episode_rewards[-1]))

/home/mark/.conda/envs/dl/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Populating replay memory...


[2017-11-04 16:47:00,222] Starting new video recorder writing to /home/mark/projects/reinforcement-learning/experiments/Breakout-v0/monitor/openaigym.video.0.14354.video000000.mp4



 Copied model parameters to target network
Step 176 (176) @ Episode 1/10000, loss: 0.0013382609467953444
Episode Reward: 0.0
Step 187 (363) @ Episode 2/10000, loss: 0.0006927750655449927
Episode Reward: 0.0
Step 164 (527) @ Episode 3/10000, loss: 0.0008092967327684164
Episode Reward: 0.0
Step 181 (708) @ Episode 4/10000, loss: 0.00071764533640816814
Episode Reward: 0.0
Step 172 (880) @ Episode 5/10000, loss: 0.00031979102641344076
Episode Reward: 0.0
Step 236 (1116) @ Episode 6/10000, loss: 0.00078760675387457017
Episode Reward: 1.0
Step 225 (1341) @ Episode 7/10000, loss: 0.00231808237731456766
Episode Reward: 1.0
Step 251 (1592) @ Episode 8/10000, loss: 6.0259255405981094e-05
Episode Reward: 1.0
Step 193 (1785) @ Episode 9/10000, loss: 0.00027064629830420017
Episode Reward: 0.0
Step 232 (2017) @ Episode 10/10000, loss: 0.03076225891709327737
Episode Reward: 1.0
Step 279 (2296) @ Episode 11/10000, loss: 0.00016380517627112567
Episode Reward: 2.0
Step 332 (2628) @ Episode 12/10000, lo

[2017-11-04 16:48:49,194] Starting new video recorder writing to /home/mark/projects/reinforcement-learning/experiments/Breakout-v0/monitor/openaigym.video.0.14354.video000050.mp4


...

Step 582 (7239999) @ Episode 9301/10000, loss: 0.0040617701597511774
 Copied model parameters to target network
Step 1383 (7240800) @ Episode 9301/10000, loss: 0.0018274032045155764
Episode Reward: 28.0
Step 914 (7241714) @ Episode 9302/10000, loss: 0.0056873476132750518
Episode Reward: 15.0
Step 1204 (7242918) @ Episode 9303/10000, loss: 0.0039437939412891865
Episode Reward: 32.0
Step 948 (7243866) @ Episode 9304/10000, loss: 0.0150764565914869376
Episode Reward: 17.0
Step 750 (7244616) @ Episode 9305/10000, loss: 0.0054957969114184385
Episode Reward: 11.0
Step 979 (7245595) @ Episode 9306/10000, loss: 0.0036785430274903774
Episode Reward: 23.0
Step 967 (7246562) @ Episode 9307/10000, loss: 0.0060701207257807255
Episode Reward: 13.0
Step 938 (7247500) @ Episode 9308/10000, loss: 0.0283583607524633472
Episode Reward: 15.0
Step 867 (7248367) @ Episode 9309/10000, loss: 0.0045471293851733214
Episode Reward: 17.0
Step 760 (7249127) @ Episode 9310/10000, loss: 0.0084251984953880313
Episode

[2017-11-05 11:24:35,113] Starting new video recorder writing to /home/mark/projects/reinforcement-learning/experiments/Breakout-v0/monitor/openaigym.video.0.14354.video009350.mp4


Step 750 (7284353) @ Episode 9351/10000, loss: 0.0020066751167178154
Episode Reward: 17.0
Step 783 (7285136) @ Episode 9352/10000, loss: 0.0036129143554717302
Episode Reward: 11.0
Step 850 (7285986) @ Episode 9353/10000, loss: 0.0051187630742788315
Episode Reward: 18.0
Step 986 (7286972) @ Episode 9354/10000, loss: 0.0047786510549485684
Episode Reward: 16.0
Step 639 (7287611) @ Episode 9355/10000, loss: 0.0051736366003751755
Episode Reward: 9.0
Step 920 (7288531) @ Episode 9356/10000, loss: 0.0044082533568143845
Episode Reward: 21.0
Step 583 (7289114) @ Episode 9357/10000, loss: 0.0093538602814078335
Episode Reward: 8.0
Step 696 (7289810) @ Episode 9358/10000, loss: 0.0120202787220478067
Episode Reward: 15.0
Step 189 (7289999) @ Episode 9359/10000, loss: 0.0030981868039816626
 Copied model parameters to target network
Step 957 (7290767) @ Episode 9359/10000, loss: 0.0086089754477143297
Episode Reward: 24.0
Step 593 (7291360) @ Episode 9360/10000, loss: 0.0042590429075062275
Episode Rew

[2017-11-05 11:30:54,277] Starting new video recorder writing to /home/mark/projects/reinforcement-learning/experiments/Breakout-v0/monitor/openaigym.video.0.14354.video009400.mp4


Step 721 (7324391) @ Episode 9401/10000, loss: 0.0066166231408715254
Episode Reward: 10.0
Step 1381 (7325772) @ Episode 9402/10000, loss: 0.0023492448963224894
Episode Reward: 29.0
Step 661 (7326433) @ Episode 9403/10000, loss: 0.0066148564219474798
Episode Reward: 10.0
Step 743 (7327176) @ Episode 9404/10000, loss: 0.0023949537426233298
Episode Reward: 12.0
Step 857 (7328033) @ Episode 9405/10000, loss: 0.0084447320550680165
Episode Reward: 17.0
Step 676 (7328709) @ Episode 9406/10000, loss: 0.0095829935744404834
Episode Reward: 10.0
Step 820 (7329529) @ Episode 9407/10000, loss: 0.0048130019567906866
Episode Reward: 13.0
Step 470 (7329999) @ Episode 9408/10000, loss: 0.0065483115613460545
 Copied model parameters to target network
Step 982 (7330511) @ Episode 9408/10000, loss: 0.0022894281428307295
Episode Reward: 16.0
Step 796 (7331307) @ Episode 9409/10000, loss: 0.0481149181723594726
Episode Reward: 9.0
Step 981 (7332288) @ Episode 9410/10000, loss: 0.0054785697720944881
Episode R